# 1 Импорты

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Embedding
from tensorflow.keras.layers import LSTM, Conv1D,Conv2D, MaxPooling1D, Flatten,Bidirectional

from tensorflow.keras.optimizers import RMSprop, SGD
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MaxAbsScaler, normalize, LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# MODEL_PATH = './models/EIDM_model.keras'
MODEL_PATH = "./models/EIDM_2.keras"

# 2 Работа с датасетом 2.0

In [ ]:
from google.colab import drive
from tqdm import tqdm
import os
drive.mount('/content/drive')
#папка для сохранения чего-то
shared_folder_path = '/content/drive/MyDrive/dudos/'

In [ ]:
def save_df( path: str, name: str):
  # Убедитесь, что папка существует
  os.makedirs(path, exist_ok=True)

  # Определите путь для сохранения файла
  file_path = os.path.join(path, name)

  # Укажите размер чанка
  chunksize = 10000

  # Откройте файл для записи
  with open(file_path, 'w') as f:
      # Запишите заголовок вручную
      df.iloc[:0].to_csv(f, index=False)

      # Создайте tqdm-объект для отслеживания прогресса
      with tqdm(total=len(df), desc="Saving to CSV", unit="row") as pbar:
          # Итерация по чанкам
          for i in range(0, len(df), chunksize):
              df_chunk = df.iloc[i:i+chunksize]
              df_chunk.to_csv(f, header=False, index=False)
              pbar.update(len(df_chunk))

  # Отображение количества значений в столбце 'Label'
  print(df['Label'].value_counts())

In [ ]:
import requests, zipfile, io
from tqdm import tqdm

In [ ]:
# URL файла
#data_url = 'http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip'
data_url = 'http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/GeneratedLabelledFlows.zip'

# Скачивание файла
response = requests.get(data_url, stream=True)
total_size = int(response.headers.get('content-length', 0))
block_size = 1024  # 1 Kilobyte
progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

# Сохранение файла временно в директорию /content
zip_path = '/content/GeneratedLabelledFlows.zip'
with open(zip_path, 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)
progress_bar.close()


100%|██████████| 284M/284M [10:02<00:00, 471kiB/s]


In [ ]:
# URL файла
#data_url = 'http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip'
data_url = 'http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/GeneratedLabelledFlows.zip'

# Скачивание файла
response = requests.get(data_url, stream=True)
total_size = int(response.headers.get('content-length', 0))
block_size = 1024  # 1 Kilobyte
progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

# Сохранение файла временно в директорию /content
zip_path = '/content/GeneratedLabelledFlows.zip'
with open(zip_path, 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)
progress_bar.close()


100%|██████████| 284M/284M [10:02<00:00, 471kiB/s]


In [ ]:
# URL файла
#data_url = 'http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip'
data_url = 'http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/GeneratedLabelledFlows.zip'

# Скачивание файла
response = requests.get(data_url, stream=True)
total_size = int(response.headers.get('content-length', 0))
block_size = 1024  # 1 Kilobyte
progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

# Сохранение файла временно в директорию /content
zip_path = '/content/GeneratedLabelledFlows.zip'
with open(zip_path, 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)
progress_bar.close()


100%|██████████| 284M/284M [10:02<00:00, 471kiB/s]


## Работа с датасетом

In [ ]:
csv_files = [
    'https://drive.google.com/file/d/1-XOXZenFNzB9a9-Id9qMnLBO4iUm7ah1/view?usp=drive_link'
    ,'https://drive.google.com/file/d/1-TJIf8I2CsEMtw38mcgzX2AKCrgs7433/view?usp=drive_link'
    ,'https://drive.google.com/file/d/1-NPAu9wANOp5W9sA5uaQd27jdFlXePYd/view?usp=drive_link'
    ,'https://drive.google.com/file/d/1-MSB7zCLk9oQPUVhItNEwffP-CazOfEL/view?usp=drive_link'
    ,'https://drive.google.com/file/d/1-HEJ7ERgsbBs5ecRnOlgqryLswpCLVxx/view?usp=drive_link'
    ,'https://drive.google.com/file/d/1-8iRULLYt2YdRVY6K4YYrcsz_qirPVma/view?usp=drive_link'
    ,'https://drive.google.com/file/d/1-7Fphn696suUEdc0s-zTZrYmCcW0mT0c/view?usp=drive_link'
    ,'https://drive.google.com/file/d/1-6XdgpLmi7x9yTrMfo-2ZaNlBDlniIXB/view?usp=drive_link'
]

# Список файлов для объединения
csv_files = [
    '/content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
    '/content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv'
    # ,'/content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv',
    ,'/content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv'
    ,'/content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv'
    ,'/content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv'
    ,'/content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv'
    # ,'/content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv'
]

# Объединение всех файлов в один DataFrame
data_frames = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)

       # Проверка наличия столбца 'Label' и унификация названий столбцов
    if 'Flow ID' not in df.columns:
        print(f"'Flow ID' column not found in {csv_file}")
    else:
        print(f"'Flow ID' column found in {csv_file}")

    data_frames.append(df)

all_data = pd.concat(data_frames, ignore_index=True)

# Проверка итогового DataFrame
print(all_data.head())
print(f'Total number of rows: {len(all_data)}')

'Flow ID' column not found in /content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
'Flow ID' column not found in /content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
'Flow ID' column not found in /content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv
'Flow ID' column not found in /content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
'Flow ID' column not found in /content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
'Flow ID' column not found in /content/drive/MyDrive/dudos/dataset/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv
    Destination Port   Flow Duration   Total Fwd Packets  \
0              54865               3                   2   
1              55054             109                   1   
2              550

In [ ]:
df = all_data
df.shape

(1947007, 79)

In [ ]:
df.columns = df.columns.str.strip()
df = df.drop(columns=['Fwd Header Length.1'])
df.shape

(1947007, 79)

In [ ]:
df['Label'].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator'],
      dtype=object)

In [ ]:
df['Label'].value_counts()

Label
BENIGN                        1643999
PortScan                       158930
DDoS                           128027
FTP-Patator                      7938
SSH-Patator                      5897
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Name: count, dtype: int64

Replace the NaN values with zeros.

In [ ]:
df.fillna(0, inplace=True)

The "Flow Bytes/s" and "Flow Packets/s" columns have non-numerical values, replace them.

In [ ]:
df.replace('Infinity', -1, inplace=True)
df[["Flow Bytes/s", "Flow Packets/s"]] = df[["Flow Bytes/s", "Flow Packets/s"]].apply(pd.to_numeric)
df.replace([np.inf, -np.inf, np.nan], -1, inplace=True)
df.shape

(1947007, 78)

Convert string characters to numbers, use LabelEncoder, not OneHotEncoder.

In [ ]:
string_features = list(df.select_dtypes(include=['object']).columns)
string_features.remove('Label')
string_features

[]

In [ ]:
le = preprocessing.LabelEncoder()
df[string_features] = df[string_features].apply(lambda col: le.fit_transform(col))

## Недостаточная выборка против дисбаланса

Набор данных несбалансирован: всего записей = 170366, «БЕЗОПАСНЫХ» записей = 168186, записей с атаками гораздо меньше: 1507 + 652 + 21 = 2180.

In [ ]:
benign_total = len(df[df['Label'] == "BENIGN"])
benign_total

1643999

In [ ]:
attack_total = len(df[df['Label'] != "BENIGN"])
attack_total

303008

In [ ]:
#df.to_csv( shared_folder_path + "web_attacks_unbalanced.csv", index=False)
save_df(shared_folder_path,  "web_attacks.csv")
df['Label'].value_counts()

Saving to CSV: 100%|██████████| 1947007/1947007 [01:23<00:00, 23232.77row/s]


Label
BENIGN                        1643999
PortScan                       158930
DDoS                           128027
FTP-Patator                      7938
SSH-Patator                      5897
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Name: count, dtype: int64


Label
BENIGN                        1643999
PortScan                       158930
DDoS                           128027
FTP-Patator                      7938
SSH-Patator                      5897
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Name: count, dtype: int64

We use **undersampling** to correct class imbalances: we remove most of the "BENIGN" records.

Form a balanced dataset web_attacks_balanced.csv in proportion: 30% attack (2180 records), 70% benign data (2180 / 30 * 70 ~ = 5087 records).

Algorithm to form a balanced df_balanced dataset:

* All the records with the attacks are copied to the new dataset.
* There are two conditions for copying "BENIGN" records to the new dataset:

     1. The next record is copyied with the benign_inc_probability.
     2. The total number of "BENIGN" records must not exceed the limit of 5087 records.

Сalculate the probability of copying a "BENIGN" record. The enlargement multiplier is used to get exactly 70% benign data (5087 records).

In [ ]:
df.shape

(1947007, 78)

In [ ]:
enlargement = 1.1
benign_included_max = attack_total / 30 * 70
benign_inc_probability = (benign_included_max / benign_total) * enlargement
print(benign_included_max, benign_inc_probability)

707018.6666666666 0.47306630559588747


Copy records from df to df_balanced, save dataset **web_attacks_balanced.csv**.

In [ ]:
import random
indexes = []
benign_included_count = 0
for index, row in df.iterrows():
    if (row['Label'] != "BENIGN"):
        indexes.append(index)
    else:
        # Copying with benign_inc_probability
        if random.random() > benign_inc_probability: continue
        # Have we achieved 70% (5087 records)?
        if benign_included_count > benign_included_max: continue
        benign_included_count += 1
        indexes.append(index)
df_balanced = df.loc[indexes]

In [ ]:
df_balanced['Label'].value_counts()

Label
BENIGN                        707019
PortScan                      158930
DDoS                          128027
FTP-Patator                     7938
SSH-Patator                     5897
Web Attack � Brute Force        1507
Web Attack � XSS                 652
Infiltration                      36
Web Attack � Sql Injection        21
Name: count, dtype: int64

If necessary, we can keep a single class of attack for experiments.

In [ ]:
df.shape

(1947007, 78)

In [ ]:
# df_balanced = df_balanced[df_balanced['Label'] != "Web Attack – Brute Force"]
# df_balanced = df_balanced[df_balanced['Label'] != "Web Attack – Sql Injection"]
# df_balanced['Label'].value_counts()

In [ ]:
#df_balanced.to_csv( shared_folder_path + "web_attacks_balanced.csv", index=False)
save_df(shared_folder_path,  "web_attacks_balanced.csv")

Saving to CSV: 100%|██████████| 1947007/1947007 [01:24<00:00, 23107.24row/s]


Label
BENIGN                        1643999
PortScan                       158930
DDoS                           128027
FTP-Patator                      7938
SSH-Patator                      5897
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Name: count, dtype: int64


In [ ]:
import gc

del [[df]]
gc.collect()
df=pd.DataFrame()

# Работа с датосетом

In [ ]:
#data_path = "./web_attacks_balanced.csv"
data_path = shared_folder_path + "web_attacks_balanced.csv"
#read dataset
data= pd.read_csv(data_path).replace([np.inf, -np.inf], np.nan).dropna(how="any")
# data = data.iloc[:, :-1]
data.shape

(1947007, 78)

In [ ]:
# Вывод первых 5 строк набора данных
print(data.head())

# Вывод информации о наборе данных
print(data.info())

   Destination Port  Flow Duration  Total Fwd Packets  Total Backward Packets  \
0             54865              3                  2                       0   
1             55054            109                  1                       1   
2             55055             52                  1                       1   
3             46236             34                  1                       1   
4             54863              3                  2                       0   

   Total Length of Fwd Packets  Total Length of Bwd Packets  \
0                           12                            0   
1                            6                            6   
2                            6                            6   
3                            6                            6   
4                           12                            0   

   Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  \
0                      6                      6            

In [ ]:
#split dataset to train and test

labels = data.pop('Label')
# Вывод уникальных значений меток
unique_labels = labels.unique()
print("Unique Labels:")
print(unique_labels)
#print(labels)
pdtrain_data, pdtest_data, pdtrain_Label, pdtest_label = train_test_split(data,labels, test_size=0.2,shuffle=True,stratify=labels)

Unique Labels:
['BENIGN' 'DDoS' 'PortScan' 'Infiltration' 'Web Attack � Brute Force'
 'Web Attack � XSS' 'Web Attack � Sql Injection' 'FTP-Patator'
 'SSH-Patator']


In [ ]:
pdtrain_Label

1898424      BENIGN
840283       BENIGN
1221086      BENIGN
1659679      BENIGN
63356          DDoS
             ...   
328791     PortScan
384976     PortScan
1772113      BENIGN
1150992      BENIGN
844763       BENIGN
Name: Label, Length: 1557605, dtype: object

In [ ]:
#convert data from pd to numpy arrary
train_data = pdtrain_data.to_numpy()
test_data = pdtest_data.to_numpy()


train_data = train_data.astype('float32')
test_data = test_data.astype('float32')


#normalize data
train_data= normalize(train_data, axis=1, norm='l1')
test_data= normalize(test_data, axis=1, norm='l1')

In [ ]:
train_data[0]

array([ 3.10082018e-04,  3.33700925e-01,  1.17012087e-05,  1.17012087e-05,
        3.62737454e-04,  7.13773712e-04,  1.81368727e-04,  1.81368727e-04,
        1.81368727e-04,  0.00000000e+00,  3.56886856e-04,  3.56886856e-04,
        3.56886856e-04,  0.00000000e+00,  1.88739095e-02,  4.10302368e-04,
        1.11233644e-01,  1.92419156e-01,  3.33420098e-01,  5.85060434e-06,
        5.85060434e-06,  5.85060434e-06,  0.00000000e+00,  5.85060434e-06,
        5.85060434e-06,  2.74978403e-04,  2.74978403e-04,  0.00000000e+00,
        2.74978403e-04,  2.74978403e-04,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  4.68048354e-04,  2.34024177e-04,
        2.05151184e-04,  2.05151184e-04,  1.81368727e-04,  3.56886856e-04,
        2.51575984e-04,  9.61352416e-05,  1.57966313e-03,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  5.85060434e-06,
        3.14469973e-04,  

In [ ]:
#reshape data to 3D (потому что в основе и первым слоем идет сверточная нейронная сеть (она работает с фото))
train_data=train_data.reshape(train_data.shape[0],train_data.shape[1])
test_data=test_data.reshape(test_data.shape[0],test_data.shape[1])

In [ ]:

#feature selection
test = SelectKBest(score_func=f_classif, k=77)
fit = test.fit(train_data, pdtrain_Label)
np.set_printoptions(precision=3)
# print(fit.scores_)

train_data = fit.transform(train_data)
# features = fit.transform(train_data)

# Summarize selected features

# train_data = features
print(test_data.shape)
test_data = fit.transform(test_data)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [31 33 55 56 57 58 59 60] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


(389402, 77)


In [ ]:
# подготовка и нормализация данных, что бы на этом можно было обучать нейронную сеть

label_encoder = LabelEncoder()
#convert string labels to int values to be able to classify them
trainVec = label_encoder.fit_transform(pdtrain_Label)
testVec = label_encoder.fit_transform(pdtest_label)

#categorize labels
train_labels = keras.utils.to_categorical(trainVec)
test_labels = keras.utils.to_categorical(testVec)

In [ ]:
# reshape data to 3D (потому что в основе и первым слоем идет сверточная нейронная сеть (она работает с фото))
train_data=train_data.reshape(train_data.shape[0],train_data.shape[1],1)
test_data=test_data.reshape(test_data.shape[0],test_data.shape[1],1)
print(train_data.shape)

(1557605, 77, 1)


In [ ]:
train_data.shape

(1557605, 77, 1)